In [6]:
import pandas as pd

data = pd.read_csv(
    "data/sp500_2015_2024_clean_tuning.csv",
    parse_dates=["Date"],
    index_col="Date"
)

print(data.shape)
print(data.head())

(1965, 14)
                  Close         High          Low         Open      Volume  \
Date                                                                         
2017-03-09  2364.870117  2369.080078  2354.540039  2363.489990  3718450000   
2017-03-10  2372.600098  2376.860107  2363.040039  2372.520020  3434400000   
2017-03-13  2373.469971  2374.419922  2368.520020  2371.560059  3166650000   
2017-03-14  2365.449951  2368.550049  2358.179932  2368.550049  3173760000   
2017-03-15  2385.260010  2390.010010  2368.939941  2370.340088  3909490000   

             Returns         MA20         MA50   Close_lag1   Close_lag5  \
Date                                                                       
2017-03-09  0.000800  2358.136011  2308.201196  2362.979980  2381.919922   
2017-03-10  0.003269  2361.372510  2310.275601  2364.870117  2383.120117   
2017-03-13  0.000367  2364.241003  2312.746602  2372.600098  2375.310059   
2017-03-14 -0.003379  2366.101001  2315.070400  2373.469971  2

In [8]:
feature_cols = ["Close", "Returns", "MA10", "MA20", "MA50", "MA100", "Close_lag1", "Close_lag5","Volume", "Vol20"]

X=data[feature_cols]
y=data["Close_tomorrow"]

In [12]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits = 5)

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

rf = RandomForestRegressor(random_state = 42, n_jobs = -1)

param_grid = {
    "n_estimators" : [100, 200, 300],
    "max_depth" : [None, 5, 10],
    "min_samples_leaf" : [1, 3, 5],
}

grid = GridSearchCV(
    estimator = rf,
    param_grid = param_grid,
    cv = tscv,
    scoring = "neg_root_mean_squared_error",
    n_jobs=-1,
    verbose=1)

grid.fit(X, y)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=RandomForestRegressor(n_jobs=-1, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [None, 5, 10],
                         'min_samples_leaf': [1, 3, 5],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_root_mean_squared_error', verbose=1)

In [18]:
print("Best param :", grid.best_params_)
print("Best CV RMSE :", -grid.best_score_)

best_rf = grid.best_estimator_

Best param : {'max_depth': None, 'min_samples_leaf': 5, 'n_estimators': 200}
Best CV RMSE : 355.5143469449128
